In [1]:
import os
import shutil
import cv2

# Set the paths
template_path = 'recc_vlog2_v2/act_g0013/act_g0013.jpg'
target_folder = 'recc_vlog'
output_folder = 'recc_vlog2_v2/act_g0013'

In [2]:
# Load the template image
template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
template_height, template_width = template.shape[:2]

In [3]:
# Initialize the counters
total_files = 0
matched_files = 0

In [4]:
# Iterate over the files in the target folder
for filename in os.listdir(target_folder):
    if filename.endswith('.jpg'):
        total_files += 1
        
        # Load the target image
        target_path = os.path.join(target_folder, filename)
        target = cv2.imread(target_path, cv2.IMREAD_GRAYSCALE)
        
        # Perform template matching
        result = cv2.matchTemplate(target, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # Check if the template is found in the target image
        if max_val >= 0.95:  # Adjust the threshold as needed
            matched_files += 1
            
            # Copy the matched file to the output folder
            output_path = os.path.join(output_folder, filename)
            shutil.copy(target_path, output_path)

            # Display the matched file name and its max_val
            print(f"Matched file: {filename} - Max correlation value: {max_val:.2f}")

# Print the summary report
print(f"Total files processed: {total_files}")
print(f"Matched files: {matched_files}")
print(f"Copied files to output folder: {matched_files}")

Matched file: frame_273.jpg - Max correlation value: 1.00
Matched file: frame_274.jpg - Max correlation value: 1.00
Matched file: frame_275.jpg - Max correlation value: 1.00
Matched file: frame_276.jpg - Max correlation value: 1.00
Matched file: frame_375.jpg - Max correlation value: 1.00
Matched file: frame_376.jpg - Max correlation value: 1.00
Matched file: frame_377.jpg - Max correlation value: 1.00
Matched file: frame_378.jpg - Max correlation value: 1.00
Total files processed: 378
Matched files: 8
Copied files to output folder: 8
